### DEPENDENCIES AND LIBRARIES

In [214]:
# !pip install python-dotenv
# !pip install tenacity
import os,json,ast
import openai
from openai import OpenAI
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_fixed

#### API Key Setup

In [66]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

#### Connectivity Test

In [74]:
client = OpenAI(api_key= api_key)
response = client.chat.completions.create(
    model = 'gpt-5-nano',
    messages = [
        {
        'role' : 'system',
        'content': 'Please act as a professional teacher, and give ans only not too much content in output.'
        },
        {
            'role': 'user', 
            'content' : 'Hi, how are you? what is 5+5 and square of that?'
        }
    ]
)

In [75]:
response.choices[0].message.content

'I’m well, thanks. 5+5 = 10; square of 10 = 100.'

### Data Loading and Cleaning

In [128]:
laptop_data = pd.read_csv('/Users/nikhilkalra/Downloads/ShopAssist_ Data + Demo/laptop_data.csv')
laptop_data.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [130]:
laptop_data['Price'] = laptop_data['Price'].str.replace(',','').str.strip().astype('int32')
laptop_data['Laptop Weight'] = laptop_data['Laptop Weight'].str.replace('kg','').str.strip().astype('float32')
laptop_data['RAM Size'] = laptop_data['RAM Size'].str.replace('GB','').str.strip().astype('int32')

In [132]:
laptop_data

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.50,Backlit Keyboard,1 year,6 hours,35000,The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.30,RGB Keyboard,2 years,4 hours,55000,The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.50,Fingerprint Sensor,3 years,8 hours,90000,The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.20,Dolby Audio,1 year,5 hours,25000,The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.80,NanoEdge Display,2 years,7 hours,200000,The ASUS ZenBook Pro is a high-end laptop that...
5,Acer,Predator,i7,Intel,2.8 GHz,16,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.20,Dual Cooling Fans,1 year,5 hours,80000,The Acer Predator is a powerhouse laptop desig...
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8,SSD,PixelSense,"13.5""",Intel Iris Plus,2256x1504,Windows 11,1.30,Touchscreen Display,1 year,10 hours,90000,The Microsoft Surface Laptop is a premium devi...
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.60,Backlit Keyboard,3 years,6 hours,60000,The Lenovo ThinkPad is a powerful laptop desig...
8,HP,Pavilion,i5,Intel,2.3 GHz,12,HDD,LCD,"15.6""",Intel UHD,1366x768,Windows 10,2.10,B&O Audio,1 year,4 hours,30000,The HP Pavilion is a budget-friendly laptop th...
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16,SSD,IPS,"17.3""",NVIDIA RTX,1920x1080,Windows 10,2.90,Aura Sync RGB Keyboard,2 years,5 hours,85000,The ASUS ROG Strix G is a high-performance gam...


### API Wrapper with Tenacity

#### API WRAPPER: Implements 'tenacity' for auto-retry logic to handle network instability robustly, making the system "Production-Grade".

In [330]:
@retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
def call_gpt(messages, tools=None):
    try:
        response = client.chat.completions.create(
            model='gpt-5-nano',
            messages=messages,
            tools=tools,
            # temperature=0,  <-- Removed as per previous fix
            # max_completion_tokens=1000 
        )

        usage = response.usage
        print(f"   [💰 Token Usage: {usage.total_tokens} (Input: {usage.prompt_tokens} + Output: {usage.completion_tokens})]")
        # -----------------------------

        return response
    except Exception as e:
        print(f"API Error!: {e}")
        raise e

### Database Query
#### TOOL IMPLEMENTATION: The specific Python logic that acts as the 'Backend API', allowing the AI to query structured data safely.

In [332]:
def get_laptops_from_db(brand = None, min_ram = None, max_price = None, max_weight = None):
    results = laptop_data.copy()
    if brand:
        results = results[results['Brand'].str.contains(brand, case = False, na = False)]
    if min_ram:
        results = results[results['RAM Size'] >= min_ram]
    if max_price:
        results = results[results['Price'] <= max_price]
    if max_weight:
        results = results[results['Laptop Weight'] <= max_weight]
    if len(results) == 0:
        return 'No laptop found'
    return results.sort_values(by = 'Price', ascending = False).head(3).to_json(orient = 'records')

In [334]:
get_laptops_from_db(brand = 'dell', max_price = 50000)

'[{"Brand":"Dell","Model Name":"Inspiron","Core":"i5","CPU Manufacturer":"Intel","Clock Speed":"2.4 GHz","RAM Size":8,"Storage Type":"SSD","Display Type":"LCD","Display Size":"15.6\\"","Graphics Processor":"Intel UHD","Screen Resolution":"1920x1080","OS":"Windows 10","Laptop Weight":2.5,"Special Features":"Backlit Keyboard","Warranty":"1 year","Average Battery Life":"6 hours","Price":35000,"Description":"The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6\\" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a 

### Tool Schema- OpenAI Function Definition
#### TOOL SCHEMA: Defines strict data types (integer, string) to eliminate parsing errors and ensure the LLM extracts accurate parameters.

In [336]:
tools = [
    {
        'type' : 'function',
        'function' : {
            'name': 'get_laptops_from_db',
            'description': 'Filters and returns laptops from the database based on the provided criteria.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'brand': {
                        'type': 'string',
                        'description': 'The brand of the laptop (e.g., Dell, HP).'
                    },
                    'min_ram': {
                        'type': 'integer',
                        'description': 'The minimum RAM size in GB.'
                    },
                    'max_price': {
                        'type': 'integer',
                        'description': 'The maximum price in INR.'
                    },
                    'max_weight': {
                        'type': 'number',
                        'description': 'The maximum weight in kg.'
                    }
                },
                'required' : []
            }
        }
        }
       ]

### Safety Check Function
#### UTILITY LAYER: Handles Safety (Moderation API) and Memory Management (Context Window Trimming) to ensure the bot is secure and efficient.

In [338]:
def check_safety(text):
    response = client.moderations.create(input=text)
    result = response.results[0].flagged
    if result == True:
        return False
    else:
        return True
    

### Conversation Optimization

In [340]:
def trim_history(history):
    if len(history) > 6:
        updated_history = [history[0]] + history[-5:]
        return updated_history
    else:
        return history

### System Prompt- Behavioral Rules
#### SYSTEM PROMPT: Enforces "Intent Confirmation" and "Strict Data Adherence" via prompt engineering to prevent hallucinations and vague searches.

In [399]:
system_message = """You are a Senior Laptop Sales Engineer. 
                    Your goal is to help customers find the perfect laptop based on their needs.
                    
                    RULES:
                    1. **INTENT CHECK:** DO NOT call the search tool immediately. First, check if the user has provided at least **2 specific preferences** (e.g., Budget, Brand, Usage, or Weight).
                       - If they have NOT provided enough detail, ask a clarifying question first.
                       - ONLY call 'get_laptops_from_db' once you have clear requirements.
                    
                    2. **STRICT DATA ADHERENCE:** - IF the tool returns 'No laptop found', state clearly that you have no matching inventory.
                       - **DO NOT** invent or suggest options (like Refurbished, Chrome OS, or external models) that are not present in the tool output.
                       - You may ask the user if they would like to adjust their criteria (e.g., increase budget).
                    
                    3. Present Top 3 options in bullet points.
                    4. Explain WHY it fits the user's needs.
                    5. Be honest about trade-offs.
                    6. Always ask a follow-up question.
                    
                    ### EXAMPLES OF CORRECT BEHAVIOR ###
                    
                    User: "Hi"
                    Assistant: "Hello! I'm here to help you find a laptop. What is your primary use case (gaming, work, study) and budget?"
                    
                    User: "I want a gaming laptop."
                    Assistant: "Great! Gaming laptops vary a lot in price. Do you have a specific budget in mind, or a preference for brands like Dell or MSI?"
                    
                    User: "Gaming laptop under 60k." (User provided 2 details: Usage + Price)
                    Assistant: (Calls tool: get_laptops_from_db(max_price=60000))
                    
                    User: "Gaming laptop under 20k."
                    Assistant: (Calls tool: get_laptops_from_db(max_price=20000))
                    Tool Output: "No laptop found"
                    Assistant: "I searched our current inventory, but unfortunately, I couldn't find any gaming laptops under 20,000 INR. Would you be open to looking at non-gaming models, or perhaps increasing the budget?"
                """


conversation_history = [{'role': 'system', 'content': system_message}]


### Orchestration- Main Chatbot Logic
#### ORCHESTRATION LAYER: Uses a two-step "ReAct" loop (Think -> Act -> Observe) to safely execute tools and handle edge cases like empty responses.

In [401]:
import json

def run_shop_assist_interactive(user_input):
    global conversation_history
    
    # 1. Safety Check (Input)
    if check_safety(user_input) == False:
        return "I cannot assist with that request."

    # 2. Update Memory & Trim
    conversation_history.append({'role': 'user', 'content': user_input})
    conversation_history = trim_history(conversation_history)
    
    # 3. First API Call
    try:
        response = call_gpt(conversation_history, tools=tools)
    except Exception:
        return "System Error: The AI service is currently unavailable. Please try again"

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    # 4. Tool Logic
    if tool_calls:
        print('   (🤖 AI searching database...)')
        conversation_history.append(response_message)
        
        for tool_call in tool_calls:
            try:
                function_args = json.loads(tool_call.function.arguments)
            except json.JSONDecodeError:
                return "Error: AI generated invalid arguments."
            
            function_response = get_laptops_from_db(
                brand=function_args.get("brand"),
                max_price=function_args.get("max_price"),
                min_ram=function_args.get("min_ram"),
                max_weight=function_args.get("max_weight")
            )
            
            conversation_history.append({
                'tool_call_id': tool_call.id,
                'role': 'tool',
                'name': tool_call.function.name,
                'content': function_response
            })
            
        # 5. Second API Call (Final Answer)
        try:
            # FIX 1: Pass tools here too for consistency
            final_response = call_gpt(conversation_history, tools=tools)
            final_text = final_response.choices[0].message.content
        except Exception:
            return "System Error: The AI service is currently unavailable. Please try again"
        
    else:
        final_text = response_message.content

    # FIX 2: Handle None/Empty responses (Crucial for gpt-5-nano)
    if final_text is None or final_text == "":
        final_text = "Sorry No Laptops found."

    # 6. Safety Check (Output)
    if check_safety(final_text) == False:
        return "My response was flagged as inappropriate."
        
    conversation_history.append({'role':'assistant', 'content':final_text})
    
    return final_text

### Interactive REPL
#### INTERACTIVE LOOP: The driver code that keeps the session alive, handles exit commands, and resets memory for clean sessions.

In [403]:
def final_assistant():
    print('ShopAssist is live, Type "exit" to close')
    print('-'*50)
    
    while True:
        user_input = input('You: ')
        if user_input.lower() in ['exit', 'end', 'bye']:
            print("ShopAssist: Goodbye!")
            break
        response = run_shop_assist_interactive(user_input)
        print(f'ShopAssist: {response}')
        print('-'*50)

### Clean State

In [405]:
# Reset Memory
conversation_history = [{'role': 'system', 'content': system_message}]
print("Memory wiped clean.")

Memory wiped clean.


### Test Cases- Validation Suite

In [206]:
final_assistant()

ShopAssist is live, Type "exit" to close
--------------------------------------------------


You:  holla my fowwer, my insta is great


ShopAssist: Nice—congrats on growing your Insta! If you want a laptop that helps with content creation (photo/video editing, posting on the go), I can pull the Top 3 options tailored to your budget and OS.

Quick questions to tailor the picks:
- Budget (in INR)?
- Preferred OS (Windows or macOS)?
- Do you need good color accuracy for editing (often a hint for a better display)?
- Priorities: portability (lighter, smaller) or performance (more RAM, faster CPU/GPU)?
- Any must-haves (SSD size, RAM, dedicated GPU, screen size)?

If you share those, I’ll fetch the best three and explain why each fits your creator workflow.
--------------------------------------------------


You:  i dont want laptop, just chatting hahahaha


ShopAssist: Haha, totally fine—chatting works too! I’m here for vibes plus help if you ever want laptop suggestions.

If you want to chat about creator gear or anything tech-related, I can help with quick tips. A few options to pick from:
- Instagram content on the go: best budget devices for editing photos/videos, lighting, and apps.
- Creator gear basics: microphones, lighting, and mobile editing workflows without breaking the bank.
- Quick tech trends for creators: what’s worth investing in now (tablets vs laptops, color-accurate screens, portability vs power).

What would you like to dive into first? And if you ever want laptop recs again, I’ll pull Top 3 with clear reasons.
--------------------------------------------------


You:  bye


ShopAssist: Goodbye!


In [298]:
final_assistant()

ShopAssist is live, Type "exit" to close
--------------------------------------------------


You:  Hello my friend


ShopAssist: Hi there! I’m glad to help you find the perfect laptop. To pull the top 3 options tailored to you, could you share a few details?

- Budget (in INR)
- Primary use (e.g., work, programming, design/creative, gaming, casual)
- Weight/portability preference (any max weight, e.g., under 1.5 kg)
- Screen size preference
- Any brand or OS preference

If you’d like, you can just say something like “under 60k for light work and video calls” and I’ll handle the rest. After you share, I’ll fetch the top 3 options and explain why each fits, including any trade-offs. 

What’s your priority right now?
--------------------------------------------------


You:  gaming laptop low budget and travel friendly


   (🤖 AI searching database...)
ShopAssist: Here are the top 3 options I found that balance gaming capability with travel-friendliness and stay within your budget range:

- Acer Swift 3 — 50,000 INR
  - Key specs: 14" IPS display, Ryzen 5 (2.3 GHz), 8GB RAM, SSD, AMD Radeon graphics, Windows 11, ~1.2 kg, up to 9 hours battery
  - Why it fits: The lightest and most affordable option in this set. Very portable for travel, enough CPU power for casual to mid-tier gaming on lower settings, and solid battery life.
  - Trade-offs: Only 8GB RAM (may need an upgrade later for heavier games), Radeon graphics aren’t as strong for modern AAA titles at high settings, 1080p display with standard color accuracy.

- Lenovo ThinkPad (60,000 INR)
  - Key specs: 14" IPS, Ryzen 7 (3.0 GHz), 16GB RAM, SSD, NVIDIA GTX, Linux OS, ~1.6 kg, ~6 hours battery
  - Why it fits: Best overall CPU and RAM for multitasking and gaming at medium settings. The 16GB RAM helps with modern titles and background workloads, a

You:  bye thanks


ShopAssist: You're welcome! If you’d like, I can save these top 3 options for quick reference later or refine them further whenever your needs change.

Would you like me to save these recommendations or adjust based on any new preferences (budget, portability, or the games you plan to play)?
--------------------------------------------------


You:  bye


ShopAssist: Goodbye!


In [300]:
final_assistant()


ShopAssist is live, Type "exit" to close
--------------------------------------------------


You:  i will kill you


ShopAssist: I cannot assist with that request.
--------------------------------------------------


You:  why not? will kill you


ShopAssist: I cannot assist with that request.
--------------------------------------------------


You:  exit


ShopAssist: Goodbye!


In [352]:
final_assistant()

ShopAssist is live, Type "exit" to close
--------------------------------------------------


You:   Holla my frind, looking for laptop for gaming with no bar at budget but travel friendly


   [💰 Token Usage: 1029 (Input: 419 + Output: 610)]
   (🤖 AI searching database...)
   [💰 Token Usage: 3819 (Input: 1431 + Output: 2388)]
ShopAssist: Here are the top 3 travel-friendly gaming options from our current inventory:

- ASUS ZenBook Pro (15.6" OLED)
  - Specs: RTX GPU, 64 GB RAM, i9 CPU, 15.6" OLED, 1.8 kg, ~7h battery, Windows 10
  - Why it fits: Very strong gaming-capable GPU and huge RAM for multitasking; superb portable OLED display; easy to carry on trips.
  - Trade-offs: Higher price; 7h battery is decent but not exceptional for long travel; 64 GB RAM is more future-proof than strictly needed for gaming.

- Dell XPS 15 (15.6" OLED)
  - Specs: GTX GPU, 32 GB RAM, i9 CPU, 15.6" OLED, 1.8 kg, ~8h battery, Windows 11
  - Why it fits: Excellent balance of gaming performance and portability; robust build and great display; best battery life among the three.
  - Trade-offs: GTX is an older GPU generation compared to RTX options; slightly less future-proof for newer titles.

-

You:  exit


ShopAssist: Goodbye!


In [407]:
final_assistant()

ShopAssist is live, Type "exit" to close
--------------------------------------------------


You:  Hi, do you have anything less than 24000?, for gaming?


   [💰 Token Usage: 1008 (Input: 660 + Output: 348)]
   (🤖 AI searching database...)
   [💰 Token Usage: 2080 (Input: 699 + Output: 1381)]
ShopAssist: I checked our current inventory for gaming laptops under 24,000 INR and I didn’t find any matches.

Would you like me to broaden the search? Here are practical options:
- Increase budget: up to around 28,000–30,000 INR to target entry-level gaming laptops with a basic discrete GPU.
- Relax gaming requirement: consider casual/indie gaming with integrated graphics (still useful for light titles).
- Narrow by other specs: if you have non-negotiables (RAM, weight, screen size, battery), I can search within a slightly wider price range that fits those needs.

Which option would you like to try, or tell me your exact preferences (brand, RAM, weight, screen size, performance targets) and I’ll tailor the search.
--------------------------------------------------


You:  no


   [💰 Token Usage: 3046 (Input: 866 + Output: 2180)]
ShopAssist: I understand. There are no gaming laptops under 24,000 INR in our current inventory.

Would you be willing to increase your budget to around 28–30k INR to include entry-level gaming laptops, or would you prefer I keep monitoring and notify you if stock changes at that price point?
--------------------------------------------------


You:  exit


ShopAssist: Goodbye!
